In [1]:
import numpy as np
import matplotlib.pylab as plt
import scipy.io
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from tensorflow import keras
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
import absl.logging

absl.logging.set_verbosity(absl.logging.ERROR) # .h5 에러메세지 무시

mat_file = scipy.io.loadmat("data/PTSDvsHC.mat")

dataset = mat_file['dataset'] 
# 90개의 데이터, 62개의 전극 채널, 1001의 타임스텝 - 1001씩 이미 에포킹을 한 상태이며, 그에따른 평균을 낸 것임(에포킹에 대해)

X = dataset
y = mat_file['Y']
y = to_categorical(y, num_classes = 2)

X = X.reshape(90, 1001, 62, 1)

X.shape

(90, 1001, 62, 1)

In [2]:
import numpy as np
import matplotlib.pylab as plt
import scipy.io
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from tensorflow import keras
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
import absl.logging
import tensorflow as tf

def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

absl.logging.set_verbosity(absl.logging.ERROR) # .h5 에러메세지 무시

mat_file = scipy.io.loadmat("data/PTSDvsHC.mat")

dataset = mat_file['dataset'] 
# 90개의 데이터, 62개의 전극 채널, 1001의 타임스텝 - 1001씩 이미 에포킹을 한 상태이며, 그에따른 평균을 낸 것임(에포킹에 대해)

subject_test_acc = []

X = dataset
y = mat_file['Y']
y = to_categorical(y, num_classes = 2)

X = X.reshape(90, 1001, 62, 1)

input_shape = (1001, 62, 1)

for i in range(len(dataset)): # loso
    X_train = np.concatenate([X[:i], X[i+1:]], axis=0)
    X_test = X[i:i+1]
    y_train = np.concatenate([y[:i], y[i+1:]], axis=0)
    y_test = y[i:i+1]

    model_input = layers.Input(shape = input_shape)

    output1 = layers.Conv2D(32, (20, 1))(model_input)
    output1 = layers.BatchNormalization()(output1)

    output2 = layers.DepthwiseConv2D((1, 62), padding = 'valid')(output1)
    output2 = layers.BatchNormalization()(output2)
    output2 = layers.Activation(mish)(output2)

    output_end = layers.AveragePooling2D(pool_size = (100, 1), strides = (15, 1))(output2)
    output_end = layers.Flatten()(output_end)
    # output_end = layers.GlobalMaxPool2D()(output3)
    
    prediction = layers.Dense(2, activation = 'softmax')(output_end)
    model = keras.Model(inputs = model_input, outputs = prediction)
    # model.summary()
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=100, batch_size=1, verbose=0)
    predictions = model.predict(X_test)
    
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(y_test, axis=1)

    acc = accuracy_score(predicted_classes, actual_classes)
    print(f"fold {i} accuracy : {acc}")

    subject_test_acc.append(acc)
    model.save(f"erp_model_0122.h5")
print(f"loso mean accuracy : {np.mean(subject_test_acc)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
fold 0 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
fold 1 accuracy : 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
fold 2 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
fold 3 accuracy : 1.0


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
fold 4 accuracy : 1.0


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
fold 5 accuracy : 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
fold 6 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
fold 7 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
fold 8 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
fold 9 accuracy : 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
fold 10 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
fold 11 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
fold 12 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
fold 13 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
fold 14 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
fold 15 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
fold 16 accuracy : 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
fold 17 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
fold 18 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
fold 19 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
fold 20 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8